In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import openmc
import os
import openmc.deplete

from IPython.display import Image
from openmc import Plot
from dotenv import load_dotenv

load_dotenv()

CROSS_SECTIONS = os.getenv('CROSS_SECTIONS')
CHAIN_FILE = os.getenv('CHAIN_FILE')

openmc.config['cross_sections'] = CROSS_SECTIONS
openmc.config['chain_file'] = CHAIN_FILE

[sebastian-TUF-Gaming-FX505GT:06761] shmem: mmap: an error occurred while determining whether or not /tmp/ompi.sebastian-TUF-Gaming-FX505GT.1000/jf.0/3771138048/shared_mem_cuda_pool.sebastian-TUF-Gaming-FX505GT could be created.
[sebastian-TUF-Gaming-FX505GT:06761] create_and_attach: unable to create shared memory BTL coordinating structure :: size 134217728 


ModuleNotFoundError: No module named 'dotenv'

In [2]:
# Define materials
# 1.6% enriched fuel
fuel = openmc.Material(name='1.6% Fuel')
fuel.set_density('g/cm3', 10.31341)
fuel.add_nuclide('U235', 3.7503e-4)
fuel.add_nuclide('U238', 2.2625e-2)
fuel.add_nuclide('O16', 4.6007e-2)
fuel.volume = 100.0

# Borated water
water = openmc.Material(name='Borated Water')
water.set_density('g/cm3', 0.740582)
water.add_nuclide('H1', 4.9457e-2)
water.add_nuclide('O16', 2.4732e-2)
water.add_nuclide('B10', 8.0042e-6)

# Zircaloy
zircaloy = openmc.Material(name='Zircaloy')
zircaloy.set_density('g/cm3', 6.55)
zircaloy.add_nuclide('Zr90', 7.2758e-3)

In [3]:
# Instantiate a Materials collection
materials = openmc.Materials([fuel, water, zircaloy])

# Export to "materials.xml"
materials.export_to_xml()

In [4]:
# Create cylinders for the fuel and clad
fuel_outer_radius = openmc.ZCylinder(x0=0.0, y0=0.0, r=0.39218)
clad_outer_radius = openmc.ZCylinder(x0=0.0, y0=0.0, r=0.45720)

# Create boundary planes to surround the geometry
min_x = openmc.XPlane(x0=-0.63, boundary_type='reflective')
max_x = openmc.XPlane(x0=+0.63, boundary_type='reflective')
min_y = openmc.YPlane(y0=-0.63, boundary_type='reflective')
max_y = openmc.YPlane(y0=+0.63, boundary_type='reflective')
min_z = openmc.ZPlane(z0=-0.63, boundary_type='reflective')
max_z = openmc.ZPlane(z0=+0.63, boundary_type='reflective')

In [5]:
# Create Universe for the fuel pin and cells
pin_cell_universe = openmc.Universe(name='1.6% Fuel Pin')

# Add Cells to the pin universe
fuel_cell = openmc.Cell(name='1.6% Fuel', fill=fuel, region=-fuel_outer_radius)
pin_cell_universe.add_cell(fuel_cell)

clad_cell = openmc.Cell(name='1.6% Clad', fill=zircaloy, region=+fuel_outer_radius & -clad_outer_radius)
pin_cell_universe.add_cell(clad_cell)

moderator_cell = openmc.Cell(name='1.6% Moderator', fill=water, region=+clad_outer_radius)
pin_cell_universe.add_cell(moderator_cell)

In [6]:
# Create root Cell and Universe
root_cell = openmc.Cell(name='root cell', fill=pin_cell_universe)
root_cell.region = +min_x & -max_x & +min_y & -max_y & +min_z & -max_z

root_universe = openmc.Universe(universe_id=0, name='root universe')
root_universe.add_cell(root_cell)

In [7]:
# Create Geometry and set root Universe
geometry = openmc.Geometry(root_universe)

In [8]:
# Export to "geometry.xml"
geometry.export_to_xml()

In [1]:
# OpenMC simulation settings
settings = openmc.Settings()
settings.batches = 100
settings.inactive = 10
settings.particles = 1000

# Create an initial uniform spatial source distribution over fissionable zones
bounds = [-0.63, -0.63, -0.63, 0.63, 0.63, 0.63]
uniform_dist = openmc.stats.Box(bounds[:3], bounds[3:], only_fissionable=True)
settings.source = openmc.IndependentSource(space=uniform_dist)

# Export to "settings.xml"
settings.export_to_xml()

NameError: name 'openmc' is not defined

In [10]:
# Set up mesh for tally
mesh = openmc.RegularMesh()
mesh.dimension = [100, 100]
mesh.lower_left = [-0.63, -0.63]
mesh.upper_right = [0.63, 0.63]

# Create mesh filter for tally
mesh_filter = openmc.MeshFilter(mesh)

In [11]:
# Create a tally to score flux and fission rate
tallies = openmc.Tallies()
tally = openmc.Tally(name='flux')
tally.filters = [mesh_filter]
tally.scores = ['flux', 'fission']
tallies.append(tally)

# Attach tallies to the model
model = openmc.Model()
model.materials = materials
model.geometry = geometry
model.tallies = tallies

In [12]:
# Export tallies to "tallies.xml"
tallies.export_to_xml()

In [13]:
# Define a voxel plot
vox_plot = openmc.Plot()
vox_plot.type = 'voxel'
vox_plot.width = (100., 100., 50.)
vox_plot.pixels = (400, 400, 200)

In [14]:
plots = openmc.Plots([vox_plot])
plots.export_to_xml()

In [ ]:
# Define depletion operator and integrate
op = openmc.deplete.CoupledOperator(model)
power = 1200.0e6  # watts
timesteps = [10.0, 10.0, 10.0]  # days
integrator = openmc.deplete.CECMIntegrator(op, timesteps, power, timestep_units='d')
integrator.integrate()

In [ ]:
# Run OpenMC!
openmc.run()

In [ ]:
results = openmc.deplete.Results("depletion_results.h5")
time, keff = results.get_keff()

# Load the statepoint file
sp = openmc.StatePoint('statepoint.100.h5')

In [ ]:
tally = sp.get_tally(scores=['flux'])
print(tally)

In [ ]:
tally.sum

In [ ]:
print(tally.mean.shape)
(tally.mean, tally.std_dev)

In [ ]:
flux = tally.get_slice(scores=['flux'])
fission = tally.get_slice(scores=['fission'])
print(flux)

In [ ]:
flux.std_dev.shape = (100, 100)
flux.mean.shape = (100, 100)
fission.std_dev.shape = (100, 100)
fission.mean.shape = (100, 100)

In [ ]:
fig = plt.subplot(121)
fig.imshow(flux.mean)
fig2 = plt.subplot(122)
fig2.imshow(fission.mean)

In [ ]:
# Determine relative error
relative_error = np.zeros_like(flux.std_dev)
nonzero = flux.mean > 0
relative_error[nonzero] = flux.std_dev[nonzero] / flux.mean[nonzero]

# distribution of relative errors
ret = plt.hist(relative_error[nonzero], bins=50)

In [ ]:
sp.source

In [ ]:
sp.source['E']

In [ ]:
# Create log-spaced energy bins from 1 keV to 10 MeV
energy_bins = np.logspace(3,7)

# Calculate pdf for source energies
probability, bin_edges = np.histogram(sp.source['E'], energy_bins, density=True)

# Make sure integrating the PDF gives us unity
print(sum(probability*np.diff(energy_bins)))

# Plot source energy PDF
plt.semilogx(energy_bins[:-1], probability*np.diff(energy_bins), drawstyle='steps')
plt.xlabel('Energy (eV)')
plt.ylabel('Probability/eV')

In [ ]:
plt.quiver(sp.source['r']['x'], sp.source['r']['y'],
           sp.source['u']['x'], sp.source['u']['y'],
           np.log(sp.source['E']), cmap='jet', scale=20.0)
plt.colorbar()
plt.xlim((-0.5,0.5))
plt.ylim((-0.5,0.5))

In [ ]:
# Close the statepoint file as a matter of best practice
sp.close()